# Class 6: The Bigger Picture

## Guest speakers

### [Ariel Kennan](https://www.arielkennan.com/)

> Ariel Kennan (she/her) is a Fellow at the [Beeck Center for Social Impact + Innovation at Georgetown University](https://beeckcenter.georgetown.edu/), where she researches and advises on best practices for using design, data, and technology for improved social safety net outcomes. Ariel leads the [Digital Benefits Network](https://beeckcenter.georgetown.edu/projects/digital-benefits-network/) to support public interest practitioners working on the social safety net through generating insights, fostering collaboration, and sparking advocacy to advance the delivery of benefits and increase economic opportunity.
>
> Previously, she served in the [New York City Mayor’s Office for Economic Opportunity](https://www.nyc.gov/site/opportunity/index.page) where she founded the nation’s first municipal [service design studio](https://www.nyc.gov/site/opportunity/portfolio/service-design-studio.page) and managed a portfolio of best-in-class digital [products](https://www.nyc.gov/site/opportunity/portfolio/products.page) to connect and deliver services. Most recently, she was the Director of Civic Innovation at [Sidewalk Labs](https://www.sidewalklabs.com/), where she led the strategy for inclusive participation in the social infrastructure of cities.

#### Questions?

## [Final Project peer grading](https://python-public-policy.afeld.me/en/nyu/final_project/peer_grading.html)

## Ask Me Anything (AMA)

Have slides on "Python beyond data analysis" as backup, but would rather talk about what you want to hear about.

## Python beyond data analysis

We've been focusing on using Python and pandas for data analysis. What else is Python used for?

### Data engineering

- Automation / recurring processes
- Copying/moving/processing/publishing data, especially Big Data
- Monitoring/alerting

### Web development

- Building web sites that are interactive (more than just content)
- Forms
- Presenting data
- Workflows, such as:
   - Signing up for things
   - Paying for things

### Machine learning

- Statistics, but _fancy_
- Building models
   - [Examples](https://tfhub.dev/)
- Finding patterns
- Recommendations
- Detection

When people say "artificial intelligence," they usually mean "machine learning."

![Diagram showing what type of machine learning may be useful, if at all](https://mitsloan.mit.edu/sites/default/files/2021-04/machine-learning-infographic_2.jpg)

[Source, with more thorough explanation](https://mitsloan.mit.edu/ideas-made-to-matter/machine-learning-explained)

#### The process

_High-level_

1. Create a model
   1. Gather a bunch of data for training
   1. If supervised machine learning, label it (give it the right answers)
   1. Segment into training and test data
   1. Train the model against the training dataset (have it identify patterns)
   1. Test the model against the test dataset
1. Run against new data
1. If reinforcement learning, model refines itself

You have a head start: [The fundamentals](https://python-public-policy.afeld.me/en/nyu/syllabus.html#learning-objectives) are applicable anywhere you're using code.

## Post-JupyterHub

- JupyterHub instance (and your copy of files) will be deleted
    - Download things you want to keep, particularly edited notebooks
- Class materials will remain available on [the site](https://python-public-policy.afeld.me/en/nyu/) and [through GitHub](https://github.com/afeld/python-public-policy/tree/nyu)
- [Running notebooks elsewhere](https://python-public-policy.afeld.me/en/nyu/resources.html#jupyter-outside-this-course)

## [Course evaluations](https://coursefeedback.nyu.edu/nyu/)

They are:

- Totally anonymous
- Not visible to me until grades are released
- A big help. Some things I took from the past:
    - Making assignments more rigorous
    - Students are hungry for more
    - People like the in-class exercises

[More info.](https://www.nyu.edu/students/student-information-and-resources/registration-records-and-graduation/final-exams-and-course-evaluations/course-evaluation.html) Please complete now, if you haven't already.

Thanks to the grader!

## Thank you!

Keep in touch:

- [Email](https://python-public-policy.afeld.me/en/nyu/syllabus.html#instructor-information)
- [@aidanfeldman](https://twitter.com/aidanfeldman/) on Twitter